# Running CEDS Scenarios

In this notebook we document how to process and run data from the CEDS database.

In [ ]:
%matplotlib inline

from os import listdir
from os.path import join, dirname
from pprint import pprint
from copy import deepcopy

import pandas as pd
import numpy as np
import pyam
from pyam.utils import LONG_IDX
import pint
from pint.pandas_interface import PintArray
from pint.errors import DimensionalityError

import pymagicc
from pymagicc.io import MAGICCData

import matplotlib.pyplot as plt
plt.style.use('bmh') 

import expectexception

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
TEST_DATA_PATH = join("..", "tests", "test_data")

## Reading in a CEDS csv

To read in CEDS csv's, we make use of the `pyam` library which is specifically designed for this purpose.

In [ ]:
def read_ceds_csv(file_to_read):
    return pyam.IamDataFrame(
        data=file_to_read,
        encoding="utf-8"
    )

ceds_pyam_df = read_ceds_csv(join(TEST_DATA_PATH, "ceds-format-example-2.csv"))
ceds_pyam_df  # this just shows the type of ceds_pyam_df
ceds_pyam_df.data.head()  # this returns the head of the underlying DataFrame

## Super brief intro to pyam

The `pyam` library provides some very natural ways of filtering their DataFrames. These are detailed in [their tutorial](https://github.com/IAMconsortium/pyam/blob/master/tutorial/pyam_first_steps.ipynb). Here we use them to help convert IAM data into the emissions variables, regions and units used by openscm and MAGICC.

In [ ]:
# this only gets first level emissions and 
# World values
tdf = ceds_pyam_df.filter(
    level=1,
    region="World",
)
tdf.variables()
tdf.regions()
tdf.data.head()

## Checking an `IamDataFrame`

It is very easy to check that the sum of a given variable's sub-categories is equal to its declared total and that the sum of regions gives the world total.

We show how in the next cell.

In [ ]:
# show check_internal_consistency method here once PR into pyam
# is accepted

## Converting CEDS data to openscm data

In the Python implementation of `openscm`, we want to work with `IamDataFrame`'s for as long as possible thanks to their useful aggregating, filtering and other methods. 

Hence all we do to convert from CEDS data to openscm is tweak the underlying `DataFrame`.

We start by aggregating the data from the CEDS csv into `openscm` relevant categories. 

We also strip 'equiv' and '-' from units so that pint can read the units and subsequently from variable names for consistency. Yes:

1. it looks odd to have HFC in units of 'CO2' but it's hopefully fairly clear that this is an equivalence unit
1. changing from IAM units and names in this way is annoying but pint is worth it so we make the switch

In [ ]:
# before we do anything, we add a metadata attribute to pyam.IamDataFrame
# which we can use as a temporary measure in this notebook
pyam.IamDataFrame.metadata = dict()

def convert_pyam_df_to_openscm_df(pyam_df):
    TMP_INDEX = ['model', 'scenario', 'region', 'year', 'unit']
    
    ceds_openscm_var_mapping = {
        "Aircraft": ["Aircraft"],
        "International Shipping": ["International Shipping"],
        "AFOLULUC": ["Agricultural Waste Burning", "Agriculture", 
                     "Forest Burning", "Grassland Burning", 
                     "Peat Burning", "Aggregate - Agriculture and LUC"],
        "Fossil": ["Energy Sector", "Industrial Sector", 
                   "Residential Commercial Other", 
                   "Solvents Production and Application", 
                   "Transportation Sector", "Waste"]
    }
    
    openscm_df = pyam.IamDataFrame(data=pyam_df.data.copy())
    
    output_df = openscm_df.filter(level=1,).data
    sectoral_df = openscm_df.filter(
        level='1-', 
        keep=False
    )
    
    handled_vars = []
    metadata = {}
    var_dfs = []
    for variable in sectoral_df.variables():
        var_bits = variable.split("|")
        base_var = "|".join(var_bits[:2])
        extension = "|".join(var_bits[2:3])
        assert "|" not in extension, "Handling more than one level deep not ready yet"
        
        for category, suffixes in ceds_openscm_var_mapping.items():
            if extension not in suffixes:
                continue
                
            openscm_var = "{}|{}".format(
                "|".join(variable.split("|")[:-1]),
                category,
            )
            
            if openscm_var in handled_vars:
                continue
            handled_vars.append(openscm_var)
            
            contrib_vars = ["{}|{}".format(base_var, s) for s in suffixes]

            var_cat_df = sectoral_df.data[sectoral_df.data.variable.isin(contrib_vars)]
            var_cat_df = pd.DataFrame(var_cat_df.groupby(TMP_INDEX).sum()['value'])
            var_cat_df = pd.concat([var_cat_df], keys=[openscm_var], names=['variable'])
            
            var_dfs.append(var_cat_df.reset_index())

            metadata[openscm_var] = "Sum of {}".format(", ".join(contrib_vars))
    
    output_df = pd.concat([output_df] + var_dfs, sort=False)
    output_df.unit = output_df.unit.str.replace("-equiv", "").str.replace("-", "")
    output_df.variable = output_df.variable.str.replace("-", "")
    
    output_df = pyam.IamDataFrame(data=output_df)
    
    output_df.metadata = pyam_df.metadata
    output_df.metadata.update(metadata)
    
    return output_df

In [ ]:
openscm_df = convert_pyam_df_to_openscm_df(ceds_pyam_df)

# check out metadata with this
print([
    "{}: {}".format(k,openscm_df.metadata[k])
    for k in list(openscm_df.metadata)[:3]
])

# check out table with this
openscm_df.head()

# check regions with this
openscm_df.regions()

# check out variables with this
openscm_df.variables()[:10]

# check out units with this
openscm_df.data.unit.unique()[:10]

## Converting units

In the next part we start to do unit conversions. Before moving on, we briefly explain how we do this.

We load units with Pint like so.

In [ ]:
ureg = pint.UnitRegistry()  # start a unit repository using the default variables
ureg.load_definitions('emissions_units.txt')  # load emissions units too
ureg._contexts  # show us which contexts we have available

In [ ]:
# define some variables with units
a = 1*ureg.C
b = 1*ureg.CO2
c = 3*ureg.N2O

In [ ]:
# they carry units with them
a
b
c

In [ ]:
# we can convert them to base units or to each other
b.to_base_units()
b.to('C')
c.to('N')

In [ ]:
# operations are units aware
a + b
a * b
(a * b).to_base_units()
a / b
(a / b).to_base_units()
a - b

In [ ]:
# with a context, we can use metric conversions to 
# do our conversions
# e.g. AR4GWP12 which is a made up metric where 1C = 20N
# hence 1 CO2 = 12/44 C = 12/44*20 N = 12/44*20*14/44 N2O

with ureg.context('AR4GWP12'):
    b
    b.to('N2O')
    12/44*20*14/44
    a.to('N') + c  # I am not sure why you need to force the conversion of `a` first...

In [ ]:
%%expect_exception DimensionalityError
# without a context to tell us about metrics, if 
# we try to do an invalid conversion, a 
# DimensionalityError will be thrown
b.to('N2O')

### Units in DataFrames

Pint comes with accesors for pandas dataframes. This is super useful but also very young so can be a bit fiddly. 

For example, below we show how to convert a variable to a given set of units whilst maintaining index etc., something which the pint accessor can't do (yet).

We will use this function to convert our data table to MAGICC units.

In [ ]:
def convert_variable_units(pyam_df, variable, target_units):
    output_df = pyam.IamDataFrame(pyam_df.data.copy())
    
    var_df = pyam_df.filter(variable=variable).data.copy()
    rest_df = pyam_df.filter(variable=variable, keep=False).data.copy()
    
    var_df = var_df.set_index(LONG_IDX).unstack(["variable", "unit"])
    var_df = var_df.pint.quantify(ureg, level=-1)
    
    for col in var_df:
        var_df[col] = var_df[col].pint.to(target_units)
    # annoying that pint dequantify destroys index
    old_index = var_df.index
    old_columns = var_df.columns
    var_df = var_df.pint.dequantify()
    var_df.index = old_index
    var_df.columns.names = old_columns.names + ['unit']

    var_df = var_df.stack().stack().reset_index()

    output_df = pyam.IamDataFrame(pd.concat([var_df, rest_df], sort=False))
    output_df.metadata = pyam_df.metadata
    return output_df

In [ ]:
print("Original DataFrame")
print("------------------")
original_df = ceds_pyam_df.filter(
    variable="Emissions|CO2*"
)
original_df.head()

print("")
print("DataFrame with converted units")
print("------------------------------")
converted_df = convert_variable_units(ceds_pyam_df, "Emissions|CO2","Gt C/yr").filter(
    variable="Emissions|CO2*"
)
converted_df.head()

print("")
print("Ratio of first line values")
print("{}".format(
    original_df.data.value.iloc[0] 
    / converted_df.data.value.iloc[0]
))

## Converting openscm data to MAGICC data

Here we show how to then convert an openscm data table to MAGICC data. MAGICC is a somewhat special beast so there's a few transformations we have to do:

- convert openscm variables to MAGICC variables and regions
    - conventionally we make everything uppercase to remind ourselves that these variables and regions will be used in FORTRAN in the end (which is case insensitive) but this might be another annoying mapping so we might take this out in future (as we're not going backwards right now, we leave it)
- convert everything to MAGICC's units

In [ ]:
def get_emissions_species(variable):
    exceptions = ["HCFC141B", "HCFC142B"]
    variable = variable.replace("_EMIS", "")
    if variable in exceptions:
        return variable
    elif variable.endswith(("I", "B")):
        return variable[:-1]
    elif variable.endswith(("AIR", "SHIP")):
        return variable.replace("AIR", "").replace("SHIP", "")
    else:
        return variable
    
def convert_openscm_to_magicc_variable(openscm_variable):
    category_codes = {
        "Aircraft": "AIR",
        "International Shipping": "SHIP",
        "AFOLULUC": "B",
        "Fossil": "I",
    }
    
    special_cases = {
        "VOC": "NMVOC",
        "Sulfur": "SOX",
        "HFC4310mee": "HFC4310",
    }
    
    # Improvement: do this with regexp
    species = openscm_variable.split("|")[1]
    if species in special_cases:
        species = special_cases[species]
        
    try:
        category = openscm_variable.split("|")[2]
        category_code = category_codes[category]
    except IndexError:
        category_code = ""
    
    return "{}{}_EMIS".format(species.upper(), category_code)
    
def convert_openscm_to_magicc_df(openscm_df):
    magicc_df = pyam.IamDataFrame(data=openscm_df.data.copy())
    magicc_df.metadata = deepcopy(openscm_df.metadata)

    magicc_df.data.variable = magicc_df.data.variable.apply(convert_openscm_to_magicc_variable)
    magicc_df.data.region = magicc_df.data.region.str.upper()
    magicc_df.data = magicc_df.filter(variable="HFC_EMIS", keep=False).data

    for variable in magicc_df.variables():
        magicc_unit_df = pymagicc.definitions.emms_units
        magicc_units = magicc_unit_df[
            magicc_unit_df["MAGICC variable"] == get_emissions_species(variable)
        ]["emissions units"].values
        assert len(magicc_units) == 1, "{} {}".format(variable, magicc_units)
        magicc_units = magicc_units[0]

        magicc_df = convert_variable_units(magicc_df, variable, magicc_units)
    
    return magicc_df

In [ ]:
magicc_df = convert_openscm_to_magicc_df(openscm_df)

# have a look at some conversions
openscm_df.data[openscm_df.data.variable.str.contains("NH3\\|Fossil")].sort_values(["region", "year"]).head()
magicc_df.data[magicc_df.data.variable.str.contains("NH3I")].sort_values(["region", "year"]).head()

openscm_df.data[openscm_df.data.variable.str.contains("CO2\\|AFOL")].sort_values(["region", "year"]).head()
magicc_df.data[magicc_df.data.variable.str.contains("CO2B")].sort_values(["region", "year"]).head()

# check out metadata with this
print([
    "{}: {}".format(k,magicc_df.metadata[k])
    for k in list(magicc_df.metadata)[:3]
])

# check out table with this
magicc_df.head()

# check out variables with this
magicc_df.variables()[:10]

# check out units with this
magicc_df.data.unit.unique()[:10]

## Supplementing CEDS data

Before we go into writing scenario files, we supplement the CEDS data. The only variable we have to do this for is C6F14 as it is required to run MAGICC6. The rest are purely for interest.

In [ ]:
c2f6_df = magicc_df.filter(variable="*C2F6*").data.copy()
c6f14_df = c2f6_df.copy()
c6f14_df.unit = "C6F14 * kt / yr"
c6f14_df.variable = "C6F14_EMIS"

# hardcoded as MAGICC7 inputs not yet public
c6f14_2015_value = 0.3500
sf = c6f14_2015_value / c2f6_df.value[c2f6_df.year == 2015][0]
c6f14_df.value *= sf

# should probably add metadata here...
magicc_df.data = pd.concat([magicc_df.data, c6f14_df], sort=False)

np.testing.assert_allclose(c6f14_df.value / c2f6_df.value, sf)

c6f14_df

In [ ]:
magicc_df.filter(
    variable=["*C2F6*", "*C6F14*"]
).line_plot(figsize=(16, 9));

## Writing MAGICC scenario files

Once we have our dataframe, we lastly want to cut it to either write SCEN or SCEN7 files.

### Converting to SCEN7 format

Firstly we show how to get things into a SCEN7 format. The major steps are:

- create a BUNKERS region from aviation and shipping emissions
- make sure we only provide the regional/sectoral breakdown for emissions if we have them, not the totals

In [ ]:
def convert_bunkers_to_magicc_variable(bunker_variable):
    return bunker_variable.replace("SHIP", "I").replace("AIR", "I")

def get_bunkers_df_from_magicc_df(magicc_df):
    ship_df = magicc_df.filter(variable="*SHIP*").data
    ship_df.variable = ship_df.variable.apply(convert_bunkers_to_magicc_variable)
    ship_df.region = "BUNKERS"
    ship_df.set_index(LONG_IDX, inplace=True)
    
    air_df = magicc_df.filter(variable="*AIR*").data
    air_df.variable = air_df.variable.apply(convert_bunkers_to_magicc_variable)
    air_df.region = "BUNKERS"
    air_df.set_index(LONG_IDX, inplace=True)

    bunkers_df = ship_df + air_df
    bunkers_df.reset_index(inplace=True)
    
    return bunkers_df

In [ ]:
def magicc_df_to_scen7_df(magicc_df):
    scen7_df = magicc_df.filter(
        variable=["*SHIP*", "*AIR*"], 
        keep=False
    )
    # strip out all the variables with breakdown
    # data available
    for variable in scen7_df.variables():
        if variable.endswith(("I_EMIS", "B_EMIS")):
            continue

        if variable.replace("_EMIS", "I_EMIS") in scen7_df.variables().tolist():
            scen7_df = scen7_df.filter(
                variable=variable, 
                keep=False
            )
    
    # to dicuss with Malte, should we do this 
    # given I don't think it matters:
    # - add in N2O breakdown
    # - add in CO2B breakdown
    
    scen7_df = scen7_df.data
    bunkers_df = get_bunkers_df_from_magicc_df(magicc_df)
    
    scen7_df = pd.concat([scen7_df, bunkers_df], sort=False)
    scen7_df = pyam.IamDataFrame(data=scen7_df)
    scen7_df.metadata = magicc_df.metadata
    
    return scen7_df

In [ ]:
scen7_df = magicc_df_to_scen7_df(magicc_df)

# check out metadata with this
print([
    "{}: {}".format(k,scen7_df.metadata[k])
    for k in list(scen7_df.metadata)[:3]
])

# check out table with this
scen7_df.head()

# check out variables with this
scen7_df.variables()[:10]

# check out regions with this
scen7_df.regions()

### Reshaping an `IamDataFrame`

Here we show how to reshape an `IamDataFrame` to get it into the format expected by `openscm` so we can then write files with the data in it.

We also tidy up the units so they look a bit nicer.

Note: we normally want to take this step last, after we have done all our aggregation etc., as it means that we no longer have an `IamDataFrame` and can't use all the helpful tools it provides any more.

In [ ]:
def tidy_units(input_unit):
    species, mass_per_time = input_unit.split("*")
    mass, time = mass_per_time.split("/")
    
    mass = mass.replace("gigametric_ton", "Gt")
    mass = mass.replace("megametric_ton", "Mt")
    mass = mass.replace("kilometric_ton", "kt")
    mass = mass.replace("metric_ton", "tt")
    
    return "{} {} / {}".format(
        mass.strip(), 
        species.strip(), 
        time.strip()
    )

def reshape_magicc_df_to_pymagicc_df(magicc_df):
    pymagicc_df = magicc_df.pivot_table(
        index=['year'], 
        columns=['model', 'scenario', 'variable', 'unit', 'region'], 
        values='value',
        aggfunc='sum',
    )
    
    years = pymagicc_df.index
    if (years % 1 == 0).all() :
        pymagicc_df.index = years.astype(int)
    pymagicc_df.index.name = "YEAR"
    
    models = pymagicc_df.columns.get_level_values("model")
    scenarios = pymagicc_df.columns.get_level_values("scenario")
    regions = pymagicc_df.columns.get_level_values("region")
    variables = pymagicc_df.columns.get_level_values("variable")
    units = pymagicc_df.columns.get_level_values("unit")
    units = [tidy_units(u) for u in units]
    todos = ["SET"] * len(units)
    
    pymagicc_df.columns = pd.MultiIndex.from_arrays(
        [models, scenarios, variables, todos, units, regions],
        names=("MODEL", "SCENARIO", "VARIABLE", "TODO", "UNITS", "REGION"),
    )
    
    pymagicc_out = MAGICCData
    pymagicc_out.df = pymagicc_df
    pymagicc_out.metadata = magicc_df.metadata
    
    return pymagicc_out

In [ ]:
pymagicc_df = reshape_magicc_df_to_pymagicc_df(scen7_df)
pymagicc_df.df

### Writing SCEN7 files

In [ ]:
label
df

In [ ]:
for label, df in pymagicc_df.df.groupby(level=["MODEL", "SCENARIO"], axis=1):        
    fn = "{}_{}".format(*label).replace(" ", "-").replace(".", "-") + ".SCEN7"

    df.columns = df.columns.droplevel("MODEL").droplevel("SCENARIO")
    
    writer = MAGICCData()
    writer.df = df
    writer.metadata = {
        "header": "Emissions scenario for {}-{}\n\n".format(*label)
    }
    writer.write(fn)

### Converting to SCEN format

The major thing here is to ensure that we only return the variables MAGICC6 uses. Otherwise very similar to SCEN7 files.

In [ ]:
def magicc_df_to_scen_df(magicc_df, world_only=True):
    if not world_only:
        raise NotImplementedError("Neccesary checks not yet included e.g. no breakdown for HFC emissions")

    scen_df = magicc_df.filter(
        variable=["*SHIP*", "*AIR*"],
        keep=False
    )
    
    scen_df = scen_df.data
    bunkers_df = get_bunkers_df_from_magicc_df(magicc_df)
    
    scen_df = pd.concat([scen_df, bunkers_df], sort=False)
    
    scen_df = pyam.IamDataFrame(data=scen_df)
    if world_only:
        scen_df = scen_df.filter(
            region="WORLD"
        )
    
    scen_emis = [
        v + "_EMIS" 
        for v in pymagicc.definitions.scen_emms_code_1
    ]
    output_df = scen_df.filter(variable=scen_emis)
    output_df.metadata = magicc_df.metadata
    
    return output_df

In [ ]:
scen_df = magicc_df_to_scen_df(magicc_df)

# check out metadata with this
print([
    "{}: {}".format(k,scen_df.metadata[k])
    for k in list(scen_df.metadata)[:3]
])

# check out table with this
scen_df.head()

# check out variables with this
scen_df.variables()[:10]

# check out regions with this
scen_df.regions()

In [ ]:
pymagicc_df = reshape_magicc_df_to_pymagicc_df(scen_df)
pymagicc_df.df

### Writing SCEN files

In [ ]:
label
df

In [ ]:
for label, df in pymagicc_df.df.groupby(level=["MODEL", "SCENARIO"], axis=1):        
    fn = "{}_{}".format(*label).replace(" ", "-").replace(".", "-") + ".SCEN"

    df.columns = df.columns.droplevel("MODEL").droplevel("SCENARIO")

    writer = MAGICCData()
    writer.df = df
    writer.metadata = {
        "header": "required for some reason\n\n"
    }
    writer.write(fn)

## Run a SCEN file we just wrote

In [ ]:
scenario = pymagicc.read_scen_file("MESSAGEix-GLOBIOM-PostParis-4-0_zero2070_5_3_nonco2_200.SCEN")
scenario

In [ ]:
results = pymagicc.run(scenario)
results['CO2I_EMIS'].plot(figsize=(16, 9))
results['SURFACE_TEMP'].plot(figsize=(16, 9));